# Kinetics II

We continue kinetic analysis, working through integral methods for batch reactor analysis, non-linear regression, and the significance of data linearization.

## For Fall 2025

Remove the extra stats parts and just focus on the visual analysis and SSE. Leave the extra statistics and confidence intervals for L30.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.stats as stats
from math import ceil, floor

## Example Problem 01

(Re)Analysis of Data Collected in a Constant Volume Batch Reactor.

We are going to continue analyzing the data set from Lecture 28.  As mentioned, inverse problems where we try to determine models and parameters from data can be tricky, and it is useful to apply several different methods of analysis if we're able. Let's recall that we're considering a single, irreversible reaction:

$$A \longrightarrow B$$

We carry out the reaction in a constant volume batch reactor, and we know the following:

\begin{align}
    C_{A0} &= 10 \ \textrm{mol} \ \textrm{L}^{-1} \\
    C_{B0} &= 0 \ \textrm{mol} \ \textrm{L}^{-1} \\
    T &= \textrm{constant}
\end{align}

We also have measured a set of concentrations (in moles per liter) at various times (in minutes).  These are plotted below. Our goal is to determine the rate law (reaction order) and rate constant for this reaction under these conditions.

In [78]:
# tdata = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30, 45, 60, 90, 120]) #time in minutes
# CAdata = np.array([10, 9.05, 8.19, 7.41, 6.70, 6.07, 5.49, 4.97, 4.49, 4.07, 3.68, 3.01, 2.23, 1.35, 0.821, 0.498, 0.111, 0.0248, 1.23E-3, 6.14E-05])

# #Concentrations in moles per liter

# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, CAdata, edgecolor = 'black', color = 'none', label = 'CA')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('CA (M)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(0, 12)
# plt.legend()
# plt.show()

### Solution to Example Problem 01

We want to know, based on that data, what is the rate law and corresponding kinetic parameters (k, reaction order, etc.) that best describe that data. With an inverse problem, we have to extract that information from the data. Since this reaction is overall (not an elementary step), at best, we can propose that the rate of reaction depends on temperature (which we capture with a rate constant) and the concentration of the reactant, A, with some unknown order, i.e., we'll propose a power law model:

$$r = k{C_A}^\alpha$$

Our analysis will start with a material balance on species A in a constant volume batch reactor:

$$\frac{dN_A}{dt} = R_AV$$

For a constant volume reactor, we can divide through by V and move it inside of the derivative:

$$\frac{dC_A}{dt} = R_A$$

And for a single reaction, we know that $R_A = -r$, so:

$$\frac{dC_A}{dt} = -r$$

We've proposed power law kinetics here, hence:

$$\frac{dC_A}{dt} = -k{C_A}^\alpha$$

At this point, we make a decision about whether we want to use a **differential** method of analysis, where we attempt to approximate reaction rates by taking numerical derivatives of data, or an **integral** method of analysis, where we attempt to solve the material balance by integration and then regress parameters in the resultant model that best describe our data.  

In **587-L28**, we used a differential method of analysis.  We showed that it is:

1. Quick to apply using finite differences
2. Convenient when you do not know the reaction order as it will be determined by regression
3. Avoids having to guess reaction orders and solve the material balance as with integral methods

On the downside, though:

1. Numerical differentiation is imprecise, and especially so when considering large changes in concentration/time
2. Imprecision arises from the assumption that the derivative is constant over the range considered, a built in feature of finite differences
3. If we aren't careful, a differential analysis can give us a false impression of true reaction kinetics, even for "perfect data" generated by a batch reactor simulation.

Today, we'll consider the integral method of analysis and highlight its strengths and weaknesses.  We'll also take this opportunity to introduce nonlinear regression, which is a very useful skill.

### Integral Analysis

As the name suggests, integral analysis of batch reactor data is based on integration, which entails solving the differential equation that comprises the batch reactor material balance.  Recall from above that we were able to develop the following ODE that describes the change in the concentration of species a with respect to time -- remember that at this point this is done symbolically for an arbitrary rate constant, $k$, and an arbitrary reaction order $\alpha$.  As of now, we do not know either of their values, so we leave them symbolic in the material balance.

$$\frac{dC_A}{dt} = -k{C_A}^\alpha$$

In a later lecture, we'll present the symbolic solution for an undetermined value of $\alpha$ and $k$, but I find it better to start with more tangible and straightforward cases to get a feel for integral analysis of data in a batch reactor. Although somewhat tedious, the most straightfoward approach is to assume a reaction order and then check to see if that model describes the measured data. As we'll see below, this can be time consuming, but it has the added advantage of letting us compare multiple models, which can help to inform our decision about which one describes our data the best.

Here, let's assume that the reaction order is either 0, 1, or 2.  Once we do that, we can solve the material balance on A to get an analytical solution for $C_A(t)$, which we can compare to our data.

#### Zero Order Solution

For a zero order reaction, the material balance on A in a constant volume reactor becomes:

$$\frac{dC_A}{dt} = -k$$

This is a separable ODE, so we'll solve it by hand 

```note
It is far better in kinetic analysis if you can obtain an analytical solution than having to resort to a numerical one.
```

$$\int_{C_{A0}}^{C_A}dC_A = \int_0^t-kdt$$

Which gives:

$$C_A = C_{A0} - kt$$

#### First Order Solution

For a first order reaction, the material balance on A in a constant volume reactor becomes:

$$\frac{dC_A}{dt} = -kC_A$$

This is a separable ODE, so we'll solve it by hand.

$$\int_{C_{A0}}^{C_A}\frac{1}{C_A}dC_A = \int_0^t-kdt$$

Which gives:

$$C_A = C_{A0}\exp\left(-kt\right)$$

#### Second Order Solution

For a second order reaction, the material balance on A in a constant volume reactor becomes:

$$\frac{dC_A}{dt} = -k{C_A}^2$$

This is a separable ODE, so we'll solve it by hand.

$$\int_{C_{A0}}^{C_A}\frac{1}{{C_A}^2}dC_A = \int_0^t-kdt$$

Which gives:

$$\frac{1}{C_A} = \frac{1}{C_{A0}} + kt$$

And, keeping with the theme of solving for $C_A(t)$, we can rearrange to get:

$$C_A = \left(\frac{1}{C_{A0}} + kt\right)^{-1}$$

To summarize, we have 3 models to compare to our data:

**Zero Order**: $C_A = C_{A0} - kt$  

**First Order**: $C_A = C_{A0}\exp\left(-kt\right)$

**Second Order**: $\frac{1}{C_A} = \frac{1}{C_{A0}} + kt$

We're going to make this comparison in two ways.  First, we will overlay these models directly with our data, which is a plot of concentration vs. time.  Then we'll vary the value of the unknown parameter (k) until we get the best fit for the data. We'll start with a very crude manual approach to get a feel for how this works, we will consider how we can do a better job of quantifying the error between measurement values and model predictions, and then we will look at more sophisticated strategies for minimizing that error.

First, a manual approach with only visual feedback about quality of fit.

In [14]:
# ##Testing zero order model
# CAZERO = lambda k, t: CA0 - k*t
# CA0    =  #mol/L
# ktest   =   #mol/L/min

# CMODZERO = CAZERO(ktest, tdata)
# CMODZERO < 0
# CMODZERO[CMODZERO < 0]
# CMODZERO[CMODZERO < 0] = 0   #Note, this is forcing the CA value to go to zero; otherwise a zero order model will go negative

# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, CAdata, edgecolor = 'black', color = 'none', label = 'CA')
# plt.plot(tdata, CMODZERO, color = 'black', linestyle = 'dashed', label = 'Zero Order Model')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('CA (M)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(0, 12)
# plt.legend()
# plt.show()

In [11]:
# ##Testing first order model
# CAONE   = lambda k, t: CA0*np.exp(-k*t)
# CA0     =  #mol/L
# ktest   =   #mol/L/min
# CMODONE = CAONE(ktest, tdata)

# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, CAdata, edgecolor = 'black', color = 'none', label = 'CA')
# plt.plot(tdata, CMODONE, color = 'black', linestyle = 'dashed', label = 'First Order Model')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('CA (M)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(0, 12)
# plt.legend()
# plt.show()

In [13]:
# ##Testing second order model
# CATWO = lambda k, t: 1/(1/CA0 + k*t)
# CA0   =  #mol/L
# ktest =   #mol/L/min

# CMODTWO = CATWO(ktest, tdata)
# plt.figure(1, figsize = (5, 5))
# plt.scatter(tdata, CAdata, edgecolor = 'black', color = 'none', label = 'CA')
# plt.plot(tdata, CMODTWO, color = 'black', linestyle = 'dashed', label = 'Second Order Model')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('CA (M)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(0, 12)
# plt.legend()
# plt.show()

The mathematically rigorous way to find the best value of the rate constant is to find the one that gives the "line of best fit."  That is to say, the best model/rate constant is the one that ***minimizes*** the total error between model predictions and measurement values.

In all of the above, the systems are very overdetermined.  We have 20 data points, and, in each of our models (zero order, first order, and second order), there is only a single unknown (k).  So there won't be a unique solution in any of these cases.  Instead, we find the solution that minimizes the error between our model prediction, which is the heart of regression analysis.

There are many ways to perform a regresion, and you have lots of choices about the nature of the objective function that you're trying to minimize.  Here, we'll keep it simple and do the most common type, which is least-squares regression. Specifically, we will quantify the total error in our system as the sum of square differences between model predictions and measurement values:  

$$SSE = \sum_i \left(y_i - \hat{y}_i\right)^2$$

Here, $y_i$ is the $i^\text{th}$ measurement of the observable $y$ at the corresponding value of independent variable, $x_i$, and $\hat{y}$ is the model prediction of the observable value ($y$) at the same condition of independent variable, $x_i$. The quantity in parentheses represents the displacement (or "error") between our model's prediction and our measurement's value.  We square it to ensure that the error is positive -- otherwise, positive and negative errors will average out to zero, making it difficult to identify the true minimum error.

In this case, we'd base our SSE on measured and model predicted values of the concentration of A since that is our observable.

$$SSE = \sum_i \left(C_{A_i} - \hat{C}_{A_i}\right)^2$$

Now that we have a lot of experience writing functions to accomplish all sorts of things, it is relatively straightforward to write a function to calculate the sum of squares for a given model at a certain value of rate constant.  We can then look at both the plot *and* the more quantitative sum of square errors for the current parameter value.

In [80]:
# def OBJZERO(k):
#     CA0    = 10 #mol/L
#     tdata  = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30, 45, 60, 90, 120]) #time in minutes
#     CAdata = np.array([10, 9.05, 8.19, 7.41, 6.70, 6.07, 5.49, 4.97, 4.49, 4.07, 3.68, 3.01, 2.23, 1.35, 0.821, 0.498, 0.111, 0.0248, 1.23E-3, 6.14E-05])
    
#     CAmodel = CAZERO(k, tdata)  #Use previously defined lambda function here instead
#     CAmodel[CAmodel < 0] = 0
#     RESID  = CAdata - CAmodel
#     SSE    = np.sum(RESID**2)
    
    # plt.figure(figsize = (5, 5))
    # plt.scatter(tdata, CAdata, edgecolor = 'black', color = 'none', label = 'CA')
    # plt.plot(tdata, CAmodel, color = 'black', linestyle = 'dashed', label = 'Second Order Model')
    # plt.xlabel('time (min)', fontsize = 12)
    # plt.ylabel('CA (M)', fontsize = 12)
    # plt.xlim(0, 120)
    # plt.ylim(0, 12)
    # plt.legend()
    # plt.show()
    
    # print(SSE)
    # return SSE

In [44]:
# OBJZERO(0.5)

#### Minimizing the error with numerical tools

It is of course better to use numerical methods to find the value of the parameter that minimizes our objective function (SSE).  We use the exact same approach for minimizing the SSE as we've been using for things like yield and reactor volume. Generally speaking, we do this with `opt.minimize_scalar()` for univariate problems and `opt.minimize()` for multivariate problems.

We'll consider the zero order model first:

In [81]:
# ans_zero = opt.minimize_scalar(OBJZERO)#, method = 'Brent', bracket = [0.01, 10])
# SSE      = ans_zero.fun
# k_zero   = ans_zero.x
# print(k_zero, SSE)

# #Calculate result for best fit zero order model
# tsmooth = np.linspace(0, max(tdata), 100)
# CA0    = 10 #mol/L
# CMODZERO = CAZERO(k_zero, tsmooth)#CA0 - k_zero*tsmooth
# CMODZERO[CMODZERO < 0] = 0

# #Overlay best fit model with data
# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, CAdata, edgecolor = 'black', color = 'none', label = 'CA')
# plt.plot(tsmooth, CMODZERO, color = 'black', linestyle = 'dashed', label = 'Zero Order Model')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('CA (M)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(0, 12)
# plt.legend()
# plt.show()

# #Overlay best fit model with data
# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, (CAdata - CAZERO(k_zero, tdata))/CAdata*100, edgecolor = 'black', color = 'none', label = 'Residual Error')
# plt.hlines(0, 0, 120, color = 'black', linestyle = 'dashed', label = 'Zero Error')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('Error (%)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(-100, 100)
# plt.legend(loc = 'upper right')
# plt.show()

#### Analysis of Zero Order Fit

Based on that result, the best fit value of the rate constant in a zero order model is 0.621, which achieves a minimum SSE of ~9.1.  Visually, it does an OK job of fitting data at low times, but it falls apart after about 10 - 15 minutes.  We notice that the zero order model over-predicts measured concentration at short times, and it underpredicts at long times.  This would be a sign that the order of reaction is too low.

Now we'll do the same optimization with a first order model.

#### First order optimization

In [82]:
# def OBJONE(k):
#     CA0     = 10 #mol/L
#     tdata   = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30, 45, 60, 90, 120]) #time in minutes
#     CAdata  = np.array([10, 9.05, 8.19, 7.41, 6.70, 6.07, 5.49, 4.97, 4.49, 4.07, 3.68, 3.01, 2.23, 1.35, 0.821, 0.498, 0.111, 0.0248, 1.23E-3, 6.14E-05])
    
#     CAmodel = CAONE(k, tdata)
#     RESID   = CAdata - CAmodel
#     SSE     = np.sum(RESID**2)
#     print(SSE)
#     return SSE

In [83]:
# #Run Optimization
# ans_one = opt.minimize_scalar(OBJONE)#, method = 'Brent', bracket = [0.01, 10])
# SSE     = ans_one.fun
# k_one   = ans_one.x
# print(k_one, SSE)

# #Calculate results for best fit first order model
# tsmooth = np.linspace(0, max(tdata), 100)
# CA0     = 10 #mol/L
# CMODONE = CAONE(k_one, tsmooth) #CA0*np.exp(-k_one*tsmooth)

# #Overlay best fit model with data
# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, CAdata, edgecolor = 'black', color = 'none', label = 'CA')
# plt.plot(tsmooth, CMODONE, color = 'black', linestyle = 'dashed', label = 'First Order Model')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('CA (M)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(0, 12)
# plt.legend()
# plt.show()

# #Overlay best fit model with data
# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, (CAdata - CAONE(k_one, tdata))/CAdata*100, edgecolor = 'black', color = 'none', label = 'Residual Error')
# plt.hlines(0, 0, 120, color = 'black', linestyle = 'dashed', label = 'Zero Error')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('Error (%)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(-1, 1)
# plt.legend(loc = 'upper right')
# plt.show()

Well, that one pretty much nails it.  It estimates a best fit rate constant of 0.1 (which is exactly what I used to generate the data), and the SSE is $10^{-4}$. It really can't get better than that, but let's see what the optimum result for second order looks like too.

#### Second order optimization

In [84]:
# def OBJTWO(k):
#     CA0     = 10 #mol/L
#     tdata   = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30, 45, 60, 90, 120]) #time in minutes
#     CAdata  = np.array([10, 9.05, 8.19, 7.41, 6.70, 6.07, 5.49, 4.97, 4.49, 4.07, 3.68, 3.01, 2.23, 1.35, 0.821, 0.498, 0.111, 0.0248, 1.23E-3, 6.14E-05])
    
#     CAmodel = CATWO(k, tdata)
#     RESID   = CAdata - CAmodel
#     SSE     = np.sum(RESID**2)
#     print(SSE)
#     return SSE

In [85]:
# #Run Optimization
# ans_two = opt.minimize_scalar(OBJTWO)#, method = 'Brent', bracket = [0.01, 10])
# SSE     = ans_two.fun
# k_two   = ans_two.x
# print(k_two, SSE)

# tsmooth = np.linspace(0, max(tdata), 100)
# CA0     = 10 #mol/L
# CMODTWO = CATWO(k_two, tsmooth) #CA0*np.exp(-k_one*tsmooth)


# #Overlay best fit model with data
# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, CAdata, edgecolor = 'black', color = 'none', label = 'CA')
# plt.plot(tsmooth, CMODTWO, color = 'black', linestyle = 'dashed', label = 'Second Order Model')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('CA (M)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(0, 12)
# plt.legend()
# plt.show()

# #Overlay best fit model with data
# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, (CAdata - CATWO(k_one, tdata))/CAdata*100, edgecolor = 'black', color = 'none', label = 'Residual Error')
# plt.hlines(0, 0, 120, color = 'black', linestyle = 'dashed', label = 'Zero Error')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('Error (%)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(-1000, 1000)
# plt.legend(loc = 'lower left')
# plt.show()

Now we have the opposite problem compared to the zero order case -- the second order model under-predicts concentration at short times, and it over-predicts their values at long times.  This is a sign that the order it too high.  Clearly, the first order model is "just right" here as it is the one that fits the data best.

Thus far, we've used "nonlinear" regression because the models that we fit for first order and second order were non-linear functions of time.  This is a powerful technique but it can also lead to problems.  Most notably, when have data that is nonlinear, it is supposed to be curved in some way.  It can be hard to tell if the amount of curvature is correct.  In this case, it is easy to see, but recall that this is actually a perfect data set that I generated by solving a first order system.  When we have messy, "real" data, it can be difficult to tell if the type of curvature observed in your model is correct, and it can be hard to resolve whether one nonlinear model is better than the other.

Where it is possible to do so, it is always useful to linearize a model.  This gains us a couple of important things.

1. Whereas deviation between experimental data and a curved line can be difficult to assess, it is relatively easy to detect deviations from linearity.  This makes linearization a powerful visual tool for assessing goodness of fit.
2. If we can linearize a model, we can use linear regression tools.  Regardless of how they are packaged (e.g., trendline in Excel or polyfit in Python), they are all based on similar methods that use linear algebra.  
3. They key here is that linear regression solutions are deterministic.  There **is** a unique solution for the parameters that give us the best fit if we use linear regression. 
4. With nonlinear regression (as above), we are forced to use interative optimization routines like `opt.minimize()` or `opt.minimize_scalar()`, which are not necessarily guaranteed to converge.  The linear algebra solutions will be much more stable, which can be very beneficial in difficult optimization problems.

For these reasons, we'll consider how to linearize these models, perform linear regressions, and visually inspect the results compared to a linearized data set.

### Linearized Models

The zero order model is easy to linearize because it is already linear in time.  Looking at it, we expect that if we plot CA vs time, the trend should be linear if the kinetics are zero order.

$$C_A = C_{A0} - kt$$

We do this below; clearly the data are not linear, but we'll work through the regression anyway just to get some practice.

In [86]:
# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, CAdata, edgecolor = 'black', color = 'none', label = 'CA')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('CA (M)', fontsize = 12)
# plt.xlim(0, 120)
# plt.ylim(0, 12)
# plt.legend()
# plt.show()

So we want to take that ostensibly linear data set (our zero order model says it is linear...) and perform linear regression on it.  We'd do this with the typical vandermonde matrix approach, where we use the model:

$$C_A = C_{A0} - kt$$

But that generally translates to the generic form with two coefficients:

$$\alpha_1t^1 + \alpha_0t^0 = C_A$$ 

We'd write a copy of that equation for every data point collected, which would give us an overdetermined system of equations (18 equations and two unknown $\alpha$'s).  Technically, we know $C_{A0}$ here, but we'll move forward treating it as an unknown coefficient in the model (the y-intercept).  That ends up in our usual matrix form:

$$XA = Y$$

Where "X" is the Vandermonde matrix truncated to include only $x^0$ and $x^1$ terms.  Y is the set of measured concentrations.  We'd solve this as usual using methods of linear algebra.

In [87]:
# X = np.vander(tdata[0:11], 2) #Note I'm truncating data to the first 10 points or so; thereafter, zero order model goes negative
# Y = CAdata[0:11]
# print(Y)
# print(X)

You should recognize the cell below as the linear regression script that returns estimates for the slope and intercept and also quantifies goodness of fit and confidence intervals. It also generates a visual representation of data fit and a plot of residual errors.

In [88]:
# ydata = CAdata[0:11]
# xdata = tdata[0:11]

# X     = np.vander(xdata, 2)#This creates the X matrix shown above, truncating at 1st order
# Y     = ydata    
# A     = np.linalg.solve(X.T@X, X.T@Y) 
# Ypred = X@A
# Ybar  = np.mean(ydata)
# Ndata = len(ydata)
# SSE   = (Y - X@A).T@(Y - X@A) #Residual sum of squares, equivalent to SSE = np.sum((Y - Ypred)**2)
# SST   = np.sum((Y - Ybar)**2) #Total sum of squares
# MSE   = SSE/Ndata             #Mean Square Error
# RMSE  = np.sqrt(MSE)          #Root Mean Square Error
# R2    = 1 - SSE/SST           #R^2
# DOF   = len(ydata) - len(A)   #Degrees of freedom = number of measurements - number of regressed parameters
# s2    = SSE/DOF               #Estimate of variance, s2 ~ σ2  
# COV   = s2*np.linalg.inv(X.T@X) #Covariance matrix, use for standard errors and conf. intervals
# m     = A[0]                  #slope of best fit line
# b     = A[1]                  #y-intercept of best fit line
# SEm   = np.sqrt(COV[0, 0])    #Standard error in slope
# SEb   = np.sqrt(COV[1, 1])    #Standard error in y-intercept
# tval  = stats.t.ppf(0.975, DOF) #t distribution value for 95% confidence intervals
# CIm   = SEm*tval              #95% confidence interval on slope
# CIb   = SEb*tval              #95% confidence interval on y-intercept
# CA0   = b
# k     = -m

# labels = ['m', 'b', 'SSE', 'SST','MSE','RMSE', 'R2', 'CA0', 'k']
# values = [m  ,  b ,  SSE ,  SST , MSE , RMSE ,  R2, CA0, k]

# print(A, '\n')

# for label, value in zip(labels, values):
#     if label == 'm':
#         print(f'{label:4s} = {value:0.2f} +/- {CIm:0.2f}')
#     elif label == 'b':
#         print(f'{label:4s} = {value:0.2f} +/- {CIb:0.2f}')
#     else:
#         print(f'{label:4s} = {value:0.2f}')

# plt.figure(figsize = (5, 5))
# plt.scatter(X[:, 0], Y, marker = 'o', color = 'none', edgecolor = 'black', label = 'experimental data')
# plt.plot(X[:, 0], Ypred, color = 'blue', linestyle = 'dashed', linewidth = 1, label = 'linear fit')
# plt.xlabel('t (min)', fontsize = 12)
# plt.ylabel('CA (mol/L)', fontsize = 12)
# # plt.xlim(floor(min(X[:, 0])), ceil(max(X[:,0])))
# # plt.ylim(floor(min(Y)), ceil(max(Y)))
# plt.legend()
# plt.show()

# plt.figure(figsize = (5, 5))
# plt.scatter(X[:, 0], (Y - Ypred)/Y*100, marker = 'o', color = 'none', edgecolor = 'black', label = 'residual error')
# plt.hlines(0, floor(min(X[:, 0])), ceil(max(X[:,0])), color = 'blue', linestyle = 'dashed', linewidth = 1, label = 'zero error')
# plt.xlabel('t (min)', fontsize = 12)
# plt.ylabel('Residual Error (%)', fontsize = 12)
# plt.title('Residual Plot')
# plt.ylim(-20, 20)
# plt.xlim(floor(min(X[:, 0])), ceil(max(X[:,0])))
# plt.legend(loc = 'upper center')
# plt.show()

Despite a pretty good $R^2$, you can see that the fit here isn't very good.  There seems to be systematic deviation in preditions--the errors are not randomly scattered around the line of best fit. This is a sign that the model we have proposed (zero order kinetics) does not adequately describe our data.

**First Order Model**

For the first order model, we have:

$$C_A = C_{A0}\exp(-kt)$$

This is a nonlinear function of, essentially kt; however, if we take the natural logarithm of both sides, we convert it to the following, which is useful:

$$\ln\left(C_A\right) = \ln\left(C_{A0}\right) - kt$$

If we plot $C_A$ against time, that should be linear if the first order model is correct.  The slope would be equal to -k, and the y-intercept would be ln(CA0).  In the matrix form:

$$\alpha_1t^1 + \alpha_0t^0 = \ln\left(C_A\right)$$

We can put this into the vandermonde form:

$$XA = Y$$

Where X is the vandermonde matrix generated from tdata (and truncated to include only zero order and first order terms), and Y is the natural log of CA.

This is plotted in the cell below, where you observe linearity.

In [89]:
# plt.figure(figsize = (5, 5))
# plt.scatter(tdata, np.log(CAdata), edgecolor = 'black', color = 'none', label = 'CA')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('ln(CA) (mol/L)', fontsize = 12)
# # plt.xlim(0, 120)
# # plt.ylim(0, 12)
# plt.legend()
# plt.show()

Now that we've visually assessed linearity, we proceed with regression analysis and return the various parameter estimates and statistics.

In [90]:
# ydata = np.log(CAdata)
# xdata = tdata

# X     = np.vander(xdata, 2)#This creates the X matrix shown above, truncating at 1st order
# Y     = ydata    
# A     = np.linalg.solve(X.T@X, X.T@Y) 
# Ypred = X@A
# Ybar  = np.mean(ydata)
# Ndata = len(ydata)
# SSE   = (Y - X@A).T@(Y - X@A) #Residual sum of squares, equivalent to SSE = np.sum((Y - Ypred)**2)
# SST   = np.sum((Y - Ybar)**2) #Total sum of squares
# MSE   = SSE/Ndata             #Mean Square Error
# RMSE  = np.sqrt(MSE)          #Root Mean Square Error
# R2    = 1 - SSE/SST           #R^2
# DOF   = len(ydata) - len(A)   #Degrees of freedom = number of measurements - number of regressed parameters
# s2    = SSE/DOF               #Estimate of variance, s2 ~ σ2  
# COV   = s2*np.linalg.inv(X.T@X) #Covariance matrix, use for standard errors and conf. intervals
# m     = A[0]                  #slope of best fit line
# b     = A[1]                  #y-intercept of best fit line
# SEm   = np.sqrt(COV[0, 0])    #Standard error in slope
# SEb   = np.sqrt(COV[1, 1])    #Standard error in y-intercept
# tval  = stats.t.ppf(0.975, DOF) #t distribution value for 95% confidence intervals
# CIm   = SEm*tval              #95% confidence interval on slope
# CIb   = SEb*tval              #95% confidence interval on y-intercept
# CA0   = np.exp(b)
# k     = -m
# k1Oa  = k
# C1Oa  = CIm
# labels = ['m', 'b', 'SSE', 'SST','MSE','RMSE', 'R2', 'CA0', 'k']
# values = [m  ,  b ,  SSE ,  SST , MSE , RMSE ,  R2, CA0, k]

# print(A, '\n')

# for label, value in zip(labels, values):
#     if label == 'm':
#         print(f'{label:4s} = {value:0.2f} +/- {CIm:0.2f}')
#     elif label == 'b':
#         print(f'{label:4s} = {value:0.2f} +/- {CIb:0.2f}')
#     else:
#         print(f'{label:4s} = {value:0.2f}')

# plt.figure(figsize = (5, 5))
# plt.scatter(X[:, 0], Y, marker = 'o', color = 'none', edgecolor = 'black', label = 'experimental data')
# plt.plot(X[:, 0], Ypred, color = 'blue', linestyle = 'dashed', linewidth = 1, label = 'linear fit')
# plt.xlabel('t (min)', fontsize = 12)
# plt.ylabel('ln(CA) (mol/L)', fontsize = 12)
# # plt.xlim(floor(min(X[:, 0])), ceil(max(X[:,0])))
# # plt.ylim(floor(min(Y)), ceil(max(Y)))
# plt.legend()
# plt.show()

# plt.figure(figsize = (5, 5))
# plt.scatter(X[:, 0], (Y - Ypred)/Y*100, marker = 'o', color = 'none', edgecolor = 'black', label = 'residual error')
# plt.hlines(0, floor(min(X[:, 0])), ceil(max(X[:,0])), color = 'blue', linestyle = 'dashed', linewidth = 1, label = 'zero error')
# plt.xlabel('t (min)', fontsize = 12)
# plt.ylabel('Residual Error (%)', fontsize = 12)
# plt.title('Residual Plot')
# plt.ylim(-1, 1)
# plt.xlim(floor(min(X[:, 0])), ceil(max(X[:,0])))
# plt.legend(loc = 'lower center')
# plt.show()

Alternatively, you can use polyfit.  You get the exact same results; I'm pretty sure it is just a convenience function that automates what we did with matrices above.  We'll use those coefficients to add the line of best fit to our linearized first order data.  You can see that it describes the data very well.

In [91]:
# coeffs = np.polyfit(tdata, np.log(CAdata), 1)
# k = -1*coeffs[0]
# CAest = np.exp(coeffs[1])
# print(k, CAest)

**Second Order Model**

For the second order model, we have:

$$\frac{1}{C_A} = \frac{1}{C_{A0}} + kt$$

We should see linearity here if we plot $\frac{1}{C_A}$ vs $t$.  See below.  Deviation from linearity is clearly observed.

In [92]:
# plt.figure(figsize = (5, 5))
# plt.scatter(tdata[:-4], 1/CAdata[:-4], edgecolor = 'black', color = 'none', label = 'CA')
# plt.xlabel('time (min)', fontsize = 12)
# plt.ylabel('1/CA (L/mol)', fontsize = 12)
# # plt.xlim(0, 120)
# # plt.ylim(0, 12)
# plt.legend()
# plt.show()

Below, I regress the coefficients for the linearized second order model using polyfit and overlay it with the "linearized" data set.

In [93]:
# ydata = 1/(CAdata[:-4]) #discard the last 4 elements; they approach infinity as CA --> 0
# xdata = tdata[:-4]

# X     = np.vander(xdata, 2)#This creates the X matrix shown above, truncating at 1st order
# Y     = ydata    
# A     = np.linalg.solve(X.T@X, X.T@Y) 
# Ypred = X@A
# Ybar  = np.mean(ydata)
# Ndata = len(ydata)
# SSE   = (Y - X@A).T@(Y - X@A) #Residual sum of squares, equivalent to SSE = np.sum((Y - Ypred)**2)
# SST   = np.sum((Y - Ybar)**2) #Total sum of squares
# MSE   = SSE/Ndata             #Mean Square Error
# RMSE  = np.sqrt(MSE)          #Root Mean Square Error
# R2    = 1 - SSE/SST           #R^2
# DOF   = len(ydata) - len(A)   #Degrees of freedom = number of measurements - number of regressed parameters
# s2    = SSE/DOF               #Estimate of variance, s2 ~ σ2  
# COV   = s2*np.linalg.inv(X.T@X) #Covariance matrix, use for standard errors and conf. intervals
# m     = A[0]                  #slope of best fit line
# b     = A[1]                  #y-intercept of best fit line
# SEm   = np.sqrt(COV[0, 0])    #Standard error in slope
# SEb   = np.sqrt(COV[1, 1])    #Standard error in y-intercept
# tval  = stats.t.ppf(0.975, DOF) #t distribution value for 95% confidence intervals
# CIm   = SEm*tval              #95% confidence interval on slope
# CIb   = SEb*tval              #95% confidence interval on y-intercept
# CA0   = 1/b
# k     = m
# labels = ['m', 'b', 'SSE', 'SST','MSE','RMSE', 'R2', 'CA0', 'k']
# values = [m  ,  b ,  SSE ,  SST , MSE , RMSE ,  R2, CA0, k]

# print(A, '\n')

# for label, value in zip(labels, values):
#     if label == 'm':
#         print(f'{label:4s} = {value:0.2f} +/- {CIm:0.2f}')
#     elif label == 'b':
#         print(f'{label:4s} = {value:0.2f} +/- {CIb:0.2f}')
#     else:
#         print(f'{label:4s} = {value:0.2f}')

# plt.figure(figsize = (5, 5))
# plt.scatter(X[:, 0], Y, marker = 'o', color = 'none', edgecolor = 'black', label = 'experimental data')
# plt.plot(X[:, 0], Ypred, color = 'blue', linestyle = 'dashed', linewidth = 1, label = 'linear fit')
# plt.xlabel('t (min)', fontsize = 12)
# plt.ylabel('1/CA (L/mol)', fontsize = 12)
# # plt.xlim(floor(min(X[:, 0])), ceil(max(X[:,0])))
# # plt.ylim(floor(min(Y)), ceil(max(Y)))
# plt.legend()
# plt.show()

# plt.figure(figsize = (5, 5))
# plt.scatter(X[:, 0], (Y - Ypred)/Y*100, marker = 'o', color = 'none', edgecolor = 'black', label = 'residual error')
# plt.hlines(0, floor(min(X[:, 0])), ceil(max(X[:,0])), color = 'blue', linestyle = 'dashed', linewidth = 1, label = 'zero error')
# plt.xlabel('t (min)', fontsize = 12)
# plt.ylabel('Residual Error (%)', fontsize = 12)
# plt.title('Residual Plot')
# plt.ylim(-100, 100)
# plt.xlim(floor(min(X[:, 0])), ceil(max(X[:,0])))
# plt.legend(loc = 'upper center')
# plt.show()

As in the zero order case, there is curvature apparent in this second order "linearization." Further, there is clear systematic deviation from the model (non-random scatter); this is a sign that the model does not describe our data very well.

### Minimizing Uncertainty by Estimating Fewer Parameters

Clearly, from all of this, we can see that the first order model fits the data best, and our regressed parameters tell us that the rate constant is 0.1 inverse minutes, which is exactly what I used to generate the data.

As a final point, some of you may question why, with our linearized first order model:

$$\ln\left(C_A\right) = \ln\left(C_{A0}\right) - kt$$

We bothered to regress the y-intercept since we already know the value of $C_{A0}$ for this experiment.  Good observation.  In general, the precision of our estimates by regression decreases as we estimate more parameters.  So, for an optimization where I already know a parameter, $C_{A0}$ in this case, it's probably best if I don't introduce uncertainty by regressing a parameter whose value I'm sure of.  How could you fix it at the known quantity?

Two ways:

1. We use the nonlinear regression approach above, since this gives us complete control over the parameters we estimate.
2. We re-work the linear form of the model and corresponding vandermonde matrix such that we don't estimate the y-intercept.  

Let's look at the latter.  Our original model was

$$\ln\left(C_A\right) = \ln\left(C_{A0}\right) - kt$$

And we wrote that as:

$$\alpha_1t^1 + \alpha_0t^0 = \ln\left(C_A\right)$$

But, since I actually know $C_{A0}$, I could instead do the following linearization.  This would allow me to regress only the rate constant.

$$\ln\left(\frac{C_A}{C_{A0}}\right) = - kt$$

In this form, I would plot $\ln\left(\frac{C_A}{C_{A0}}\right)$ vs. t.  In matrix form, this becomes:

$$\alpha_1t^1 = \ln\left(\frac{C_A}{C_{A0}}\right)$$

$$XA = Y$$

Where X is a Vandermonde matrix created from tdata; however, we only keep the first order powers of time here (since there is no zero order coefficient in the model).  Y, in this case, is $\ln\left(\frac{C_A}{C_{A0}}\right)$, and A is only a single coefficient, namely the slope of the line.  See below for implementation.  We can't use polyfit here, but if we know how to work with the vandermonde matrix, we have a lot of flexibility in regressions.  This is similar to what you can do with the LINEST function in Excel.

In [94]:
# CA0   = 10.0
# X     = tdata.reshape(len(tdata), 1)
# Y     = np.log(CAdata/CA0)
# A     = np.linalg.solve(X.T@X, X.T@Y) 
# Ypred = X@A
# Ybar  = np.mean(ydata)
# Ndata = len(ydata)
# SSE   = (Y - X@A).T@(Y - X@A) #Residual sum of squares, equivalent to SSE = np.sum((Y - Ypred)**2)
# SST   = np.sum((Y - Ybar)**2) #Total sum of squares
# MSE   = SSE/Ndata             #Mean Square Error
# RMSE  = np.sqrt(MSE)          #Root Mean Square Error
# R2    = 1 - SSE/SST           #R^2
# DOF   = len(ydata) - len(A)   #Degrees of freedom = number of measurements - number of regressed parameters
# s2    = SSE/DOF               #Estimate of variance, s2 ~ σ2  
# COV   = s2*np.linalg.inv(X.T@X) #Covariance matrix, use for standard errors and conf. intervals
# m     = A[0]                  #slope of best fit line
# SEm   = np.sqrt(COV[0, 0])    #Standard error in slope
# tval  = stats.t.ppf(0.975, DOF) #t distribution value for 95% confidence intervals
# CIm   = SEm*tval              #95% confidence interval on slope
# k     = -m
# k1Ob  = k
# C1Ob  = CIm
# labels = ['m', 'SSE', 'SST','MSE','RMSE', 'R2', 'k']
# values = [m  ,  SSE ,  SST , MSE , RMSE ,  R2, k]

# print(A, '\n')

# for label, value in zip(labels, values):
#     if label == 'm':
#         print(f'{label:4s} = {value:0.2f} +/- {CIm:0.2f}')
#     elif label == 'b':
#         print(f'{label:4s} = {value:0.2f} +/- {CIb:0.2f}')
#     else:
#         print(f'{label:4s} = {value:0.2f}')

# plt.figure(figsize = (5, 5))
# plt.scatter(X[:, 0], Y, marker = 'o', color = 'none', edgecolor = 'black', label = 'experimental data')
# plt.plot(X[:, 0], Ypred, color = 'blue', linestyle = 'dashed', linewidth = 1, label = 'linear fit')
# plt.xlabel('t (min)', fontsize = 12)
# plt.ylabel('ln(CA/CA0)', fontsize = 12)
# # plt.xlim(floor(min(X[:, 0])), ceil(max(X[:,0])))
# # plt.ylim(floor(min(Y)), ceil(max(Y)))
# plt.legend()
# plt.show()

# plt.figure(figsize = (5, 5))
# plt.scatter(X[:, 0], (Y - Ypred)/Y*100, marker = 'o', color = 'none', edgecolor = 'black', label = 'residual error')
# plt.hlines(0, floor(min(X[:, 0])), ceil(max(X[:,0])), color = 'blue', linestyle = 'dashed', linewidth = 1, label = 'zero error')
# plt.xlabel('t (min)', fontsize = 12)
# plt.ylabel('Residual Error (%)', fontsize = 12)
# plt.title('Residual Plot')
# #plt.ylim(-0.3, 0.3)
# plt.xlim(floor(min(X[:, 0])), ceil(max(X[:,0])))
# plt.legend(loc = 'lower center')
# plt.show()

In [95]:
# print(f'If we regress the slope and the intercept, we get k = {k1Oa:10.8f} +/- {C1Oa:10.8f}')
# print(f'If we regress the slope not the intercept, we get k = {k1Ob:10.8f} +/- {C1Ob:10.8f}')